In [2]:
pip install tensorflow opencv-python numpy pandas matplotlib flask



     ---------------------------------------- 0.0/48.6 kB ? eta -:--:--
     ---------------------------------------- 48.6/48.6 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/390.2 MB 3.7 MB/s eta 0:01:46
   ---------------------------------------- 0.4/390.2 MB 3.7 MB/s eta 0:01:46
   ---------------------------------------- 0.5/390.2 MB 4.1 MB/s eta 0:01:36
   ---------------------------------------- 0.6/390.2 MB 3.5 MB/s eta 0:01:53
   ---------------------------------------- 0.6/390.2 MB 3.1 MB/s eta 0:02:06
   ---------------------------------------- 0.7/390.2 MB 2.3 MB/s eta 0:02:51
   ---------------------------------------- 0.8/390.2 MB 2.7 MB/s eta 0:02:27
   ---------------------------------------- 0.9/390.2 MB 2.4 MB/s eta 0:02:43
   ---------------------------------------- 0.9/390.2 MB 2.4 MB/s eta 0:02:43
   ---------------------------------------- 0.9/390.2 MB 2.0 MB/s eta 0:03

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    r'E:\skin-cancer-detection\data\train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    r'E:\skin-cancer-detection\data\train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

# Model building (using transfer learning)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet')

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

# Training
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

# Save model
model.save('model/skin_cancer_model.h5')

Found 7684 images belonging to 2 classes.
Found 1921 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 370s 2s/step - accuracy: 0.7311 - loss: 0.5058 - val_accuracy: 0.8667 - val_loss: 0.3138
Epoch 2/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 211s 875ms/step - accuracy: 0.8750 - loss: 0.3052 - val_accuracy: 0.8870 - val_loss: 0.2934
Epoch 3/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 198s 822ms/step - accuracy: 0.8824 - loss: 0.2832 - val_accuracy: 0.8907 - val_loss: 0.2811
Epoch 4/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 180s 746ms/step - accuracy: 0.8818 - loss: 0.2801 - val_accuracy: 0.8844 - val_loss: 0.2791
Epoch 5/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 222s 923ms/step - accuracy: 0.8872 - loss: 0.2722 - val_accuracy: 0.8933 - val_loss: 0.2744
Epoch 6/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 215s 893ms/step - accuracy: 0.8913 - loss: 0.2582 - val_accuracy: 0.8943 - val_loss: 0.2625
Epoch 7/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 202s 838ms/step - accuracy: 0.8841 - loss: 0.2684 - val_accuracy: 0.8964 - val_loss: 0.2701
Epoch 8/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 189s 785ms/step - accuracy: 0.8925 - lo

In [2]:
# Evaluation
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    r"E:\skin-cancer-detection\data\test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy*100:.2f}%')

# Confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix

predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)

cm = confusion_matrix(test_generator.classes, predicted_classes)
print("Confusion Matrix:")
print(cm)

Found 1000 images belonging to 2 classes.
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 834ms/step - accuracy: 0.9161 - loss: 0.2380
Test accuracy: 91.20%
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 404ms/step
Confusion Matrix:
[[271 229]
 [283 217]]
